In [3]:
# !pip install keras
# !pip install tensorflow
!pip install xgboost

     |████████████████████████████████| 166.7 MB 13 kB/s /s eta 0:00:01


In [4]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import accuracy_score

# from numpy import array
# from keras.models import Sequential
# from keras.layers import LSTM
# from keras.layers import Dense
# from keras.layers import Dropout
#from sklearn

In [5]:
xls = pd.ExcelFile('2 years data.xlsx',engine='openpyxl')
df1 = pd.read_excel(xls, 'Planilha1')
df2 = pd.read_excel(xls, 'Price increase dates')
df3 = pd.read_excel(xls, 'Mandatory vacation dates',names=['start','end'])

In [6]:
df1 = df1.iloc[:245815]
df2.dropna(axis=1,inplace=True)
df3.dropna(inplace=True)
df3.drop([0],inplace=True)

In [7]:
df1.drop(['Order nº','Custumer Name','Item Name','Subgroup_description'],axis=1,inplace=True)
df2.drop(['Subgroup of products (description)'],axis=1,inplace=True)

In [8]:
df1

,Order date,Custumer Id,State,City,Country,Item Id,Quantity,Subgruop_ID
0,2019-10-01,195.0,SP,CAMPINAS,BRASIL,30510018.0,3.0,51.0
1,2019-10-01,195.0,SP,CAMPINAS,BRASIL,30510019.0,2.0,51.0
2,2019-10-01,195.0,SP,CAMPINAS,BRASIL,30600083.0,10.0,60.0
3,2019-10-01,195.0,SP,CAMPINAS,BRASIL,30610099.0,1.0,61.0
4,2019-10-01,195.0,SP,CAMPINAS,BRASIL,30610071.0,1.0,61.0
...,...,...,...,...,...,...,...,...
245810,2021-09-13,5000.0,EX,MONTEVIDEO,URUGUAI,30510074.0,40.0,51.0
245811,2021-09-13,5000.0,EX,MONTEVIDEO,URUGUAI,30510075.0,80.0,51.0
245812,2021-09-13,5000.0,EX,MONTEVIDEO,URUGUAI,30510189.0,20.0,51.0
245813,2021-09-13,5000.0,EX,MONTEVIDEO,URUGUAI,30530005.0,20.0,53.0


In [9]:
le = preprocessing.LabelEncoder()
for col in ['Country','State','City']:
    df1[col] = le.fit_transform(df1[[col]])

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [10]:
df = df1.groupby(['Order date','Custumer Id','Item Id','Country','State','City','Subgruop_ID']).sum().reset_index()

In [11]:
df

,Order date,Custumer Id,Item Id,Country,State,City,Subgruop_ID,Quantity
0,2019-10-01,62.0,30510001.0,1,12,42,51.0,50.0
1,2019-10-01,62.0,30510004.0,1,12,42,51.0,6.0
2,2019-10-01,62.0,30510007.0,1,12,42,51.0,3.0
3,2019-10-01,62.0,30510026.0,1,12,42,51.0,10.0
4,2019-10-01,62.0,30520004.0,1,12,42,52.0,50.0
...,...,...,...,...,...,...,...,...
244855,2021-09-13,5000.0,30710008.0,8,8,126,71.0,10.0
244856,2021-09-13,5000.0,30710011.0,8,8,126,71.0,30.0
244857,2021-09-13,5000.0,30710014.0,8,8,126,71.0,5.0
244858,2021-09-13,8112.0,30060030.0,1,23,36,6.0,4000.0


In [12]:
cus_clus = []
for cusid in df['Custumer Id'].unique():
    new_data = pd.DataFrame(pd.date_range(df['Order date'].min(),df['Order date'].max(),freq='d'),columns=['Date'])
    new_data = pd.merge(new_data,df.loc[df['Custumer Id']==cusid][['Order date']].drop_duplicates(),
                        left_on='Date',right_on='Order date',how='outer')
    new_data['Buy']= new_data['Order date'].isna()==0
    new_data['Buy'].replace({True:1,False:0},inplace=True)
    new_data.drop(['Order date'],axis=1,inplace=True)

    new_data = pd.merge(new_data,df2[['Date of price increase']].drop_duplicates(),
                        left_on='Date',right_on='Date of price increase',how='outer')
    new_data['Price increase']= new_data['Date of price increase'].isna()==0
    new_data['Price increase'].replace({True:1,False:0},inplace=True)
    new_data.drop(['Date of price increase'],axis=1,inplace=True)

    new_data['Vacation'] = 0
    for i in range(1,1+len(df3)):
        new_data = pd.merge(new_data,pd.DataFrame(pd.date_range(df3['start'][i],df3['end'][i],freq='d'),columns=['vDate']),left_on='Date',right_on='vDate',how='outer')
        new_data['vDate']= new_data[['vDate']].isna()==0
        new_data['vDate'].replace({True:1,False:0},inplace=True)
        new_data['Vacation'] = new_data['Vacation'] + new_data['vDate']
        new_data.drop(['vDate'],axis=1,inplace=True)

    cus_clus.append(new_data.Buy.to_numpy())

wcss = []
for i in range(1, 11): 
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(cus_clus) 
    wcss.append(kmeans.inertia_)

plt.plot(range(1, 11), wcss)
plt.xlabel('Number of clusters')
plt.ylabel('WCSS') 
plt.show()

In [13]:
n_clus = 1
km = KMeans(n_clusters = n_clus)
km.fit(cus_clus)

metf = []
for k in range(n_clus):
    met = []
    for cusid in df['Custumer Id'].unique():

        new_data = pd.DataFrame(pd.date_range(df['Order date'].min(),df['Order date'].max(),freq='d'),columns=['Date'])
        new_data = pd.merge(new_data,df.loc[df['Custumer Id']==cusid][['Order date']].drop_duplicates(),
                            left_on='Date',right_on='Order date',how='outer')
        new_data['Buy']= new_data['Order date'].isna()==0
        new_data['Buy'].replace({True:1,False:0},inplace=True)
        new_data.drop(['Order date'],axis=1,inplace=True)

        new_data = pd.merge(new_data,df2[['Date of price increase']].drop_duplicates(),
                            left_on='Date',right_on='Date of price increase',how='outer')
        new_data['Price increase']= new_data['Date of price increase'].isna()==0
        new_data['Price increase'].replace({True:1,False:0},inplace=True)
        new_data.drop(['Date of price increase'],axis=1,inplace=True)

        new_data['Vacation'] = 0
        for i in range(1,1+len(df3)):
            new_data = pd.merge(new_data,pd.DataFrame(pd.date_range(df3['start'][i],df3['end'][i],freq='d'),columns=['vDate']),left_on='Date',right_on='vDate',how='outer')
            new_data['vDate']= new_data[['vDate']].isna()==0
            new_data['vDate'].replace({True:1,False:0},inplace=True)
            new_data['Vacation'] = new_data['Vacation'] + new_data['vDate']
            new_data.drop(['vDate'],axis=1,inplace=True)

        new_data.drop(['Date'],axis=1,inplace=True)




        if km.predict(new_data.Buy.to_numpy().reshape(1,-1))==k:

            back = 90
            ahead = 30

            x = np.array([new_data['Buy'][i:i+back].to_numpy() for i in range(len(new_data)-back+1)])

            y = []
            for i in range(len(new_data)-back+1):
                ar = np.where(new_data.Buy[back+i:back+90+i].to_numpy()==1)[0]
                if len(ar)==0:
                    y.append(0)
                else:
                    y.append(1)
            y = np.array(y)





            model = xgb.XGBClassifier()
            model.fit(x[:550], y[:550],verbose=False)
            met.append(accuracy_score(model.predict(x[550:]),y[550:]))
    metf.append([sum(met)/len(met),len(met)])

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:57:54] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:57:54] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:57:54] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/ec2-user/anaconda3/envs/python3/lib/pyth

[12:57:54] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:57:54] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:57:55] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/ec2-user/anaconda3/envs/python3/lib/pyth

...

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/ec2-user/anaconda3/envs/python3/lib/pyth

[12:58:51] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:58:51] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:58:51] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/ec2-user/anaconda3/envs/python3/lib/pyth

[12:58:51] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:58:51] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:58:51] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:58:51] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [14]:
metf

[[0.6778947368421051, 551]]

n_steps_in = 90
n_features = 2
n_steps_out= 30

def split_sequences(sequences, n_steps_in=300, n_steps_out=60):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, 1:], sequences[end_ix:out_end_ix, 0]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

x,y = split_sequences(new_data.to_numpy(), n_steps_in = 90, n_steps_out= 90)

y_n = []
for i in range(len(y)):
    y_n.append(np.where(y[i]==1)[0][0])

len(y),len(x)

model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps_in, 2)))
#model.add(Dropout(0.2))
model.add(LSTM(10))
#model.add(Dropout(0.2))
model.add(Dense(1, activation='relu',))
model.compile(optimizer='rmsprop', loss='mse',)
# fit model
model.fit(x, np.array(y_n), epochs=10, verbose=0)
print(model.summary())
# scores = model.evaluate(X_test, y_test, verbose=0)
# print("Accuracy: %.2f%%" % (scores[1]*100))

model.predict(x[10].reshape(-1,90,2))

y_n[50]